## Goal - Count the number of gloms/crypts missed in predicted mask which are there in groundtruth; the number of gloms predicted extra which aren't there in the groundtruth; and the number gloms predicted wrongly. 
<hr>


### Data - 
1. *Kidney Data* - 
Kidney data has 5 public test images and 10 private test images. We have 5 such models. ((5+10) * 5 = 75) The experiments need to be replicated for 75 pairs of ground truth and predicted images for kidney data.
2. *Colon Data* - 
Colon data has 2 test images. We have 15 models (1. Kidney model generalized on colon data; 2. Model trained only on colon data; 3. Model trained on both kidney and colon data). (2*15 = 30) The experiments need to be replicated for 30 pairs of ground truth and predicted images for colon data.


### Approach 1 - Naive approach - 

1. Count the number of gloms in ground truth json masks.
2. Convert the predicted rle into json format, count the number of gloms in the predicted mask.
3. Take the difference between the two.

This gives us the difference in number of gloms in the real and predicted mask, but doesn't tell us specifically the number of gloms that were missed, predicted extra or wrongly predicted. We have the code for this ready and tested. (Link - https://github.com/cns-iu/HuBMAP---Hacking-the-Kidney/blob/main/Result%20Analysis/Gloms%20missed.ipynb)


### Approach 2 - Human Inspection on Overlayed Masks - 

1. Convert the ground truth json mask into 2D numpy array.
2. Plot the numpy array of ground truth mask and save it.
3. Similarly, convert the predicted rle into json format and then into 2D numpy array.
4. Plot the numpy array of predicted mask and save it. 
5. Overlay groundtruth mask and predicted mask in photoshop and perform human inspection to find the number of gloms missed, predicted extra and wrongly predicted.

This approach will give us accurate results, but is human supervised. This approach has been implemented for one pair of images, needs to be done for rest of the images. 

### Approach 3 - Find the common area between corresponding gloms - 

1. Each mask can be represented as a list of gloms, where each glom is represented by the list of coordinates that make the glom and each coordinate is represented by a (x,y) coordinate pair. Therefore, each mask is a 3D list of the following dimensions - ( number of gloms * number of coordinates in each glom * 2 ). We have 2 such lists, one for ground truth mask (gt mask) and predicted mask. 
2. To calculate the missed gloms, for each glom in the gt mask, find the corresponding mask in the predicted gloms.
3. To find the corresponding masks, we can count the number of same coordinates in each glom, if it is above a certain threshold (say 80%), we will consider it a match.
4. If such a glom is not found, add it to the missed gloms. *But if we find such a glom, should we take it out of the ground truth glom list? In case of overlapping gloms, two or more gt gloms may try to get matched with one predicted glom.* As per the discussions done earlier, we leave it in the list, even if 2 gloms match (to a certain degree) to one glom in predicted mask, we will not count them as missed.
5. To calculate the gloms predicted that were not there in gt mask, repeat step 2,3,4 with roles of gt mask and predicted masks reversed.
6. To calculate wrongly predicted gloms, the threshold for the match can be tweaked. 

This approach will likely give accurate results but is computation extensive (on GPU shouldnt be a problem though), the code for this is under development. 

### Approach 4 - Sliding window approach - 

1. Slide with a n * n window over both the ground truth mask and predicted mask. n should approximately be equal to the diameter of the gloms.
2. If a glom is found in gt, try to find an overlap in predicted masks.
3. If a glom is in gt but not in predicted, add it to extra predicted gloms, if a glom is in predicted not in gt, add it to missed gloms. 

This approach will be faster as compared to approach 3. 
